In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import math
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import top_k_accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt

In [3]:
# Load the model before making predictions
model = load_model('nepali_language_model.h5')

In [4]:
#Version 2
def predict_next_words(model, tokenizer, text, max_sequence_len, top_n=3):
    # Start with the seed text
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')

    # Predict the next word probabilities
    predicted_probs = model.predict(token_list, verbose=0)

    # Get the indices of the top N words with the highest probability
    predicted_word_indices = np.argsort(predicted_probs[0])[::-1][:top_n]

    # Map the predicted indices back to words
    predicted_words = []
    for index in predicted_word_indices:
        for word, idx in tokenizer.word_index.items():
            if idx == index:
                predicted_words.append(word)
                break

    return predicted_words


In [10]:
import pickle

# Load the tokenizer from the file
with open('tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

print("Tokenizer loaded successfully!")


Tokenizer loaded successfully!


In [11]:
#Version 2
def predict_while_typing():
    sentence = ""
    print("Start typing (Type 'q' to quit): ")

    while True:
        # Take the user input (next word)
        input_text = input("Type next word: ")

        # Exit condition
        if input_text.lower() == 'q':
            break

        # Add typed input to the sentence
        sentence += " " + input_text

        # Predict the next words based on the current sentence
        predicted_words = predict_next_words(model, tokenizer, sentence, 12, top_n=3)

        # Display the predicted words and ask for the user's choice
        print(f"Predicted words:")
        for i, word in enumerate(predicted_words, 1):
            print(f"{i}. {word}")

        # Ask the user to select a word
        try:
            choice = int(input(f"Choose a word by number (1, 2, or 3): "))

            if choice not in [1, 2, 3]:
                print("Invalid choice. Please choose 1, 2, or 3.")
                continue  # Skip this loop iteration and let the user try again

            # Add the chosen word to the sentence
            sentence += " " + predicted_words[choice - 1]

        except ValueError:
            print("Please enter a valid number (1, 2, or 3).")
            continue  # Skip this loop iteration and let the user try again

        # Display the current sentence (typed + predicted)
        print(f"Current sentence: {sentence}")

# Run the real-time prediction
predict_while_typing()


Start typing (Type 'q' to quit): 
Type next word: यस प्रोजेक्टको
Predicted words:
1. लागि
2. धेरै
3. हामीले
Choose a word by number (1, 2, or 3): 1
Current sentence:  यस प्रोजेक्टको लागि
Type next word: हामीले धेरै मेहनत
Predicted words:
1. गर्नुपर्‍यो।
2. मेहनत
3. यसलाई
Choose a word by number (1, 2, or 3): 1
Current sentence:  यस प्रोजेक्टको लागि हामीले धेरै मेहनत गर्नुपर्‍यो।
Type next word: q
